In [2]:
import os, sys, inspect
import pandas as pd
import numpy as np
from datetime import date
from collections import OrderedDict

import sqlite3
import psycopg2
from psycopg2 import Error

import param
import hvplot
import hvplot.pandas
import holoviews as hv
from holoviews import opts
import panel as pn

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML, clear_output, display_html
import ipywidgets as widgets
from qgrid import show_grid

display(HTML(data="""
<style>
    div#notebook-container     {width: 80%;}
    div#menubar-container      {width: 65%;}
    div#maintoolbar-container  {width: 60%;}
</style>
"""))

In [3]:
import src
from src.analysis.processing import Analysis

In [4]:
analysis = Analysis()

In [13]:
display(Markdown('<h2>Customer Segmentation Analysis</h2>'))
sections = ["Customer Data"]
analysis_section = ["Description"]

sub_sections = ["Data Summary", "Missing Analysis", "Outlier Analysis", "Statistical Study"]

accordions = OrderedDict()
accordions["Loading"] = widgets.Accordion(children=[widgets.Output() for section in sections])
[accordions["Loading"].set_title(i, section) for i, section in enumerate(sections)]

accordions["Project Summary"] = widgets.Accordion(children=[widgets.Output() for section in analysis_section])
[accordions["Project Summary"].set_title(i, section) for i, section in enumerate(analysis_section)]

for section in sections:
    accordions[section] = widgets.Accordion(children=[widgets.Output() for sub_section in sub_sections])
    [accordions[section].set_title(i, sub_section) for i, sub_section in enumerate(sub_sections)]
    
widget_fields = widgets.Tab(children=[accordions[t] for t in accordions])
[widget_fields.set_title(i, sub) for i, sub in enumerate(accordions.keys())]

<h2>Customer Segmentation Analysis</h2>

[None, None, None]

In [14]:
widget_fields

In [17]:
section = "Customer Data"

with accordions["Loading"].children[sections.index(section)]:
    clear_output()
    display(Markdown("<h2> Initiating data loading ... </h2>"))
    analysis.get_data()
    

with accordions["Project Summary"].children[sections.index(section)]:
    clear_output()
    display(Markdown("<h2> Kaggle Challenge - Instacart Customer Analysis & Segmentation </h2>"))
    display(Markdown(r'''<p align="center">
                         <img width="1300" height="100" src="https://miro.medium.com/max/1160/1*yf7Bk7LpZCH5wcIGSxBqjA.png"></p>'''))
    
    
    
with accordions[section].children[0]:
    clear_output()
    display(Markdown(r'<h2> Overview of Data </h2>'))
    display(analysis.grid_df_display([analysis.descriptive_data(analysis.data['customer_data'][analysis.vars(['Customer'])]),
                                     analysis.data_type_analysis(analysis.data['customer_data'])]))
    
    customer_data = analysis.data["customer_data"].head(10000)
    table = analysis.holoview_table(df=customer_data, column=analysis.vars(['Customer']), width=1500, height=300)
    display(pn.Row(
        pn.Column("Dataframe of <code><b><font size='+2'>Insta-Cart Customer Data</font></b></code>", table)
    ))

## SQL

In [9]:
config = dict(
    host = "hansken.db.elephantsql.com",
    port = 5432,
    user = "wxznjcvj",
    dbname = "wxznjcvj",
    password = "h9IRl7iLUHn7JayUX-A6QqPvjHw9IkSP",
)

%store config

Stored 'config' (dict)


In [10]:
config

{'host': 'hansken.db.elephantsql.com',
 'port': 5432,
 'user': 'wxznjcvj',
 'dbname': 'wxznjcvj',
 'password': 'h9IRl7iLUHn7JayUX-A6QqPvjHw9IkSP'}

In [11]:
conn = psycopg2.connect(**config)
cursor = conn.cursor() # allows Python code to execute PostgreSQL command in a database session
print(conn.get_dsn_parameters(),"\n")

{'user': 'wxznjcvj', 'dbname': 'wxznjcvj', 'host': 'hansken.db.elephantsql.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [13]:
def execute(q, fetch_all=False):
    """Create an execute function with re-generates cursor if was previously
    corrupted due to error
    """
    global conn
    global cursor
    try:
        cursor.execute(q)
    except psycopg2.errors.InFailedSqlTransaction:
        conn.commit()
    if fetch_all:
        return cursor.fetchall()
    return cursor

In [14]:
customer_data = analysis.data["customer_data"]

In [15]:
query = '''
DROP TABLE IF EXISTS customer_data;

CREATE TABLE customer_data(
    id                    SERIAL PRIMARY KEY,
    order_id              INT,
    product_id            INT,
    add_to_cart_order     INT,
    reordered             INT,
    user_id               INT,
    eval_set              TEXT,
    order_number          INT,
    order_day_of_week     INT,
    order_hour_of_day     INT,
    days_since_last_order FLOAT,
    product_name          TEXT,
    aisle_id              INT,
    aisle                 TEXT,
    product_group_id      INT,
    product_group         TEXT
);
'''

execute(query)
conn.commit()
print("Table created successfully in PostgreSQL ...")

Table created successfully in PostgreSQL ...


In [16]:
postgreSQL_select_Query = "select * from customer_data"

cursor.execute(postgreSQL_select_Query)
print("Selecting rows from mobile table using cursor.fetchall")
customers_records = cursor.fetchall() 

Selecting rows from mobile table using cursor.fetchall


In [ ]:
print("Print each row and it's columns values")
for row in customers_records:
    print(row)

In [ ]:
query = '''
INSERT INTO customer_data (order_id, product_id, add_to_cart_order, reordered, user_id, eval_set, order_number, order_day_of_week, order_hour_of_day, days_since_last_order, product_name, aisle_id, aisle, product_group_id, product_group) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

customer_values = customer_data.values
cursor.executemany(query, customer_values)
conn.commit()

In [ ]:
query = '''
SELECT * FROM customer_data
'''
execute(query)

rows = cursor.fetchall()
for row in rows:
    print(row)

## Questions:
### Resources:
***
1. Features Explanation: [Information on each features]('https://www.lexjansen.com/sesug/2019/SESUG2019_Paper-252_Final_PDF.pdf')